In [6]:
from zhipuai import ZhipuAI
import time

class ZhiPu():
    def __init__(self, api_key, model_index):
        self.model_map = {
            1: "GLM-4-0520",
            2: "GLM-4-Plus",
            3: "GLM-4-Air",
            4: "glm-4v-plus",
            5: "glm-4v",
        }
        self.api_key = api_key
        self.index = model_index
        
    # 这里添加调用视频理解接口的方法
    
# 使用示例
api_key = '16f16231dfe4305998fd75e46327303e.jPHIpoatD80fJw7H'
model_index = 4  # 假设您选择使用GLM-4V-Plus模型
zhipu = ZhiPu(api_key, model_index)
# 调用zhipu对象的方法来实现视频理解功能

In [4]:
import requests
import json

# 智谱AI开放平台的API URL
url = "https://bigmodel.cn/usercenter/apikeys"

# 构建请求数据
data = {
    "model": "glm-4v",  # 指定模型为GLM-4V-Plus
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "请总结以下视频内容"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://your-video-url.com"  # 视频URL
                    }
                }
            ]
        }
    ],
    "max_tokens": 1024,  # 最大返回的token数
    "temperature": 0.5  # 温度参数，控制输出的随机性
}

# 设置请求头
headers = {'Content-Type': 'application/json'}

# 将数据转换为JSON格式
data_json = json.dumps(data)

# 发送POST请求
response = requests.post(url, data=data_json, headers=headers)

# 检查响应状态码
if response.status_code == 200:
    print(response.json())  # 打印视频内容识别结果
else:
    print(f"Error: {response.status_code}, {response.text}")

Error: 405, <html>
<head><title>405 Not Allowed</title></head>
<body>
<center><h1>405 Not Allowed</h1></center>
<hr><center>nginx</center>
</body>
</html>



In [12]:
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
import requests
from io import BytesIO
from modelscope import snapshot_download
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from PIL import Image

# 模型和设备配置
model_dir = snapshot_download('ZhipuAI/glm-4v-9b')
device = "auto"
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
glm4_vl = AutoModelForCausalLM.from_pretrained(model_dir, device_map=device, trust_remote_code=True, torch_dtype=torch.float16).eval()

app = FastAPI()

class ChatCompletionRequest(BaseModel):
    model: str
    messages: list
    max_tokens: int = 50
    temperature: float = 1.0

def generate_text(model: str, messages: list, max_tokens: int, temperature: float):
    query = messages[0]["content"][0]["text"]
    image_url = messages[0]["content"][1]["image_url"]["url"]
    image_get_response = requests.get(image_url)
    image = None
    if image_get_response.status_code == 200:
        image = Image.open(BytesIO(image_get_response.content)).convert("RGB")
    else:
        print("Failed to download image")
    print(query, image_url, image)
    response, history = glm4_vl.chat(tokenizer, image=image, query=query, history=None, temperature=temperature, max_length=max_tokens)
    return response

@app.post("/v1/chat/completions")
async def create_chat_completion(request: ChatCompletionRequest):
    response = generate_text(request.model, request.messages, request.max_tokens, request.temperature)
    return {"choices": [{"message": {"content": response}}], "model": request.model}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8888)

KeyboardInterrupt: 

In [15]:
import requests
import json

url = "https://tv.sohu.com/v/dXMvMzM1OTQyNzA5LzQwNDI3ODIyMS5zaHRtbA==.html"
data = {
    "model": "glm-4v",
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "这是什么？"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://tv.sohu.com/v/dXMvMzM1OTQyNzA5LzQwNDI3ODIyMS5zaHRtbA==.html"
                    }
                }
            ]
        }
    ],
    "max_tokens": 1024,
    "temperature": 0.5
}
headers = {'Content-Type': 'application/json'}
data_json = json.dumps(data)
response = requests.post(url, data=data_json, headers=headers)
if response.status_code == 200:
    print(response.json())
else:
    print(f"Error: {response.status_code}, {response.text}")

JSONDecodeError: Expecting value: line 2 column 1 (char 2)